## Einbettungen

In unserem vorherigen Beispiel haben wir mit hochdimensionalen Bag-of-Words-Vektoren der Länge `vocab_size` gearbeitet und die niedrigdimensionalen Positionsdarstellungsvektoren explizit in eine spärliche One-Hot-Darstellung umgewandelt. Diese One-Hot-Darstellung ist nicht speichereffizient. Außerdem wird jedes Wort unabhängig von den anderen behandelt, sodass One-Hot-codierte Vektoren keine semantischen Ähnlichkeiten zwischen Wörtern ausdrücken.

In dieser Einheit werden wir weiterhin den **News AG**-Datensatz untersuchen. Zu Beginn laden wir die Daten und holen einige Definitionen aus der vorherigen Einheit.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Was ist ein Embedding?

Die Idee eines **Embeddings** besteht darin, Wörter mithilfe von niedrigdimensionalen, dichten Vektoren darzustellen, die die semantische Bedeutung des Wortes widerspiegeln. Später werden wir besprechen, wie man sinnvolle Wort-Embeddings erstellt, aber vorerst betrachten wir Embeddings einfach als eine Möglichkeit, die Dimensionalität eines Wortvektors zu reduzieren.

Eine Embedding-Schicht nimmt also ein Wort als Eingabe und erzeugt einen Ausgabevektor mit einer festgelegten `embedding_size`. In gewisser Weise ähnelt sie einer `Dense`-Schicht, aber anstatt einen One-Hot-codierten Vektor als Eingabe zu verwenden, kann sie eine Wortnummer verarbeiten.

Indem wir eine Embedding-Schicht als erste Schicht in unserem Netzwerk verwenden, können wir von einem Bag-of-Words-Modell zu einem **Embedding-Bag**-Modell wechseln. Dabei wird jedes Wort in unserem Text zunächst in das entsprechende Embedding umgewandelt, und anschließend wird eine Aggregationsfunktion über alle diese Embeddings berechnet, wie z. B. `sum`, `average` oder `max`.

![Bild, das einen Embedding-Klassifikator für fünf Sequenzwörter zeigt.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Unser Klassifikator-Neuronales-Netzwerk besteht aus den folgenden Schichten:

* `TextVectorization`-Schicht, die einen String als Eingabe nimmt und einen Tensor mit Token-Nummern erzeugt. Wir werden eine angemessene Vokabulargröße `vocab_size` festlegen und weniger häufig verwendete Wörter ignorieren. Die Eingabeform wird 1 sein, und die Ausgabeform wird $n$ sein, da wir $n$ Token als Ergebnis erhalten, von denen jedes Zahlen von 0 bis `vocab_size` enthält.
* `Embedding`-Schicht, die $n$ Zahlen nimmt und jede Zahl in einen dichten Vektor einer bestimmten Länge (in unserem Beispiel 100) reduziert. Der Eingabetensor der Form $n$ wird somit in einen $n\times 100$-Tensor umgewandelt.
* Aggregationsschicht, die den Durchschnitt dieses Tensors entlang der ersten Achse berechnet, d. h. sie berechnet den Durchschnitt aller $n$ Eingabetensoren, die verschiedenen Wörtern entsprechen. Um diese Schicht zu implementieren, verwenden wir eine `Lambda`-Schicht und übergeben ihr die Funktion zur Berechnung des Durchschnitts. Die Ausgabe wird die Form 100 haben und die numerische Darstellung der gesamten Eingabesequenz sein.
* Abschließender `Dense`-linearer Klassifikator.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Im `summary`-Ausdruck entspricht die erste Tensor-Dimension `None` in der **output shape**-Spalte der Minibatch-Größe, und die zweite entspricht der Länge der Token-Sequenz. Alle Token-Sequenzen im Minibatch haben unterschiedliche Längen. Wir werden im nächsten Abschnitt besprechen, wie man damit umgeht.

Jetzt lass uns das Netzwerk trainieren:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Hinweis**: Wir erstellen den Vektorisierer basierend auf einem Teil der Daten. Dies wird durchgeführt, um den Prozess zu beschleunigen, und es könnte dazu führen, dass nicht alle Token aus unserem Text im Vokabular enthalten sind. In diesem Fall würden diese Token ignoriert, was zu einer leicht geringeren Genauigkeit führen kann. Allerdings liefert ein Teil des Textes in der Praxis oft eine gute Schätzung des Vokabulars.


### Umgang mit variablen Sequenzgrößen

Lassen Sie uns verstehen, wie das Training in Minibatches abläuft. Im obigen Beispiel hat der Eingabetensor die Dimension 1, und wir verwenden 128 lange Minibatches, sodass die tatsächliche Größe des Tensors $128 \times 1$ beträgt. Allerdings ist die Anzahl der Tokens in jedem Satz unterschiedlich. Wenn wir die `TextVectorization`-Schicht auf eine einzelne Eingabe anwenden, ist die Anzahl der zurückgegebenen Tokens unterschiedlich, abhängig davon, wie der Text tokenisiert wird:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Wenn wir den Vektorisierer jedoch auf mehrere Sequenzen anwenden, muss er einen Tensor mit rechteckiger Form erzeugen, sodass er nicht verwendete Elemente mit dem PAD-Token (was in unserem Fall null ist) auffüllt:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Hier können wir die Einbettungen sehen:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Hinweis**: Um die Menge an Auffüllung zu minimieren, kann es in einigen Fällen sinnvoll sein, alle Sequenzen im Datensatz in der Reihenfolge zunehmender Länge (oder genauer gesagt, Anzahl der Token) zu sortieren. Dies stellt sicher, dass jede Minibatch Sequenzen ähnlicher Länge enthält.


## Semantische Einbettungen: Word2Vec

In unserem vorherigen Beispiel hat die Einbettungsschicht gelernt, Wörter in Vektorrepräsentationen abzubilden, jedoch hatten diese Repräsentationen keine semantische Bedeutung. Es wäre wünschenswert, eine Vektorrepräsentation zu erlernen, bei der ähnliche Wörter oder Synonyme Vektoren entsprechen, die in Bezug auf eine Vektordistanz (zum Beispiel euklidische Distanz) nahe beieinander liegen.

Um dies zu erreichen, müssen wir unser Einbettungsmodell mit einer großen Textsammlung vortrainieren, indem wir eine Technik wie [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) verwenden. Diese basiert auf zwei Hauptarchitekturen, die verwendet werden, um eine verteilte Repräsentation von Wörtern zu erzeugen:

 - **Continuous bag-of-words** (CBoW), bei dem wir das Modell darauf trainieren, ein Wort aus dem umgebenden Kontext vorherzusagen. Gegeben ist das N-Gramm $(W_{-2},W_{-1},W_0,W_1,W_2)$, und das Ziel des Modells ist es, $W_0$ aus $(W_{-2},W_{-1},W_1,W_2)$ vorherzusagen.
 - **Continuous skip-gram** ist das Gegenteil von CBoW. Das Modell verwendet das umgebende Fenster von Kontextwörtern, um das aktuelle Wort vorherzusagen.

CBoW ist schneller, während Skip-Gram zwar langsamer ist, aber eine bessere Repräsentation für seltene Wörter liefert.

![Bild, das sowohl die CBoW- als auch die Skip-Gram-Algorithmen zur Umwandlung von Wörtern in Vektoren zeigt.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Um mit der Word2Vec-Einbettung, die auf dem Google-News-Datensatz vortrainiert wurde, zu experimentieren, können wir die **gensim**-Bibliothek verwenden. Unten finden wir die Wörter, die 'neural' am ähnlichsten sind.

> **Hinweis:** Wenn Sie zum ersten Mal Wortvektoren erstellen, kann das Herunterladen einige Zeit in Anspruch nehmen!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Wir können auch die Vektoreinbettung aus dem Wort extrahieren, um sie beim Training des Klassifikationsmodells zu verwenden. Die Einbettung hat 300 Komponenten, aber hier zeigen wir aus Gründen der Klarheit nur die ersten 20 Komponenten des Vektors:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Das Großartige an semantischen Einbettungen ist, dass man die Vektor-Codierung basierend auf Semantik manipulieren kann. Zum Beispiel können wir nach einem Wort suchen, dessen Vektorrepräsentation so nah wie möglich an den Wörtern *König* und *Frau* ist und so weit wie möglich vom Wort *Mann* entfernt ist:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Ein Beispiel oben verwendet einige interne GenSym-Magie, aber die zugrunde liegende Logik ist tatsächlich ziemlich einfach. Eine interessante Sache an Einbettungen ist, dass man normale Vektoroperationen auf Einbettungsvektoren durchführen kann, und das würde Operationen auf Wort**bedeutungen** widerspiegeln. Das obige Beispiel kann in Form von Vektoroperationen ausgedrückt werden: Wir berechnen den Vektor, der **KÖNIG-MANN+FRAU** entspricht (Operationen `+` und `-` werden auf Vektordarstellungen der entsprechenden Wörter durchgeführt), und finden dann das nächstgelegene Wort im Wörterbuch zu diesem Vektor:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Wir mussten kleine Koeffizienten zu den *man*- und *woman*-Vektoren hinzufügen – versuchen Sie, diese zu entfernen, um zu sehen, was passiert.

Um den nächstgelegenen Vektor zu finden, verwenden wir TensorFlow-Mechanismen, um einen Vektor von Abständen zwischen unserem Vektor und allen Vektoren im Vokabular zu berechnen, und finden dann den Index des minimalen Wortes mit `argmin`.


Während Word2Vec eine großartige Möglichkeit zu sein scheint, die Semantik von Wörtern auszudrücken, hat es viele Nachteile, darunter die folgenden:

* Sowohl CBoW- als auch Skip-Gram-Modelle sind **prädiktive Einbettungen** und berücksichtigen nur den lokalen Kontext. Word2Vec nutzt den globalen Kontext nicht.
* Word2Vec berücksichtigt nicht die **Morphologie** von Wörtern, d. h. die Tatsache, dass die Bedeutung eines Wortes von verschiedenen Teilen des Wortes, wie z. B. dem Stamm, abhängen kann.

**FastText** versucht, die zweite Einschränkung zu überwinden, und baut auf Word2Vec auf, indem es Vektordarstellungen für jedes Wort und die Zeichen-n-Gramme innerhalb jedes Wortes lernt. Die Werte der Darstellungen werden dann bei jedem Trainingsschritt zu einem Vektor gemittelt. Obwohl dies eine Menge zusätzlicher Berechnungen beim Pretraining erfordert, ermöglicht es den Wort-Einbettungen, Subwort-Informationen zu kodieren.

Eine andere Methode, **GloVe**, verwendet einen anderen Ansatz für Wort-Einbettungen, der auf der Faktorisierung der Wort-Kontext-Matrix basiert. Zunächst wird eine große Matrix erstellt, die die Anzahl der Wortvorkommen in verschiedenen Kontexten zählt, und dann wird versucht, diese Matrix in niedrigeren Dimensionen so darzustellen, dass der Rekonstruktionsverlust minimiert wird.

Die gensim-Bibliothek unterstützt diese Wort-Einbettungen, und Sie können mit ihnen experimentieren, indem Sie den Modell-Ladecode oben ändern.


## Verwendung vortrainierter Embeddings in Keras

Wir können das obige Beispiel anpassen, um die Matrix in unserer Embedding-Schicht mit semantischen Embeddings wie Word2Vec vorab zu füllen. Die Vokabulare des vortrainierten Embeddings und des Textkorpus werden wahrscheinlich nicht übereinstimmen, daher müssen wir eines auswählen. Hier untersuchen wir die beiden möglichen Optionen: die Verwendung des Tokenizer-Vokabulars und die Verwendung des Vokabulars aus den Word2Vec-Embeddings.

### Verwendung des Tokenizer-Vokabulars

Bei der Verwendung des Tokenizer-Vokabulars haben einige Wörter aus dem Vokabular entsprechende Word2Vec-Embeddings, während andere fehlen. Angenommen, unsere Vokabulargröße ist `vocab_size` und die Länge des Word2Vec-Embedding-Vektors ist `embed_size`, wird die Embedding-Schicht durch eine Gewichtsmatrix der Form `vocab_size`$\times$`embed_size` dargestellt. Wir füllen diese Matrix, indem wir das Vokabular durchgehen:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Für Wörter, die nicht im Word2Vec-Wortschatz vorhanden sind, können wir sie entweder als Nullen belassen oder einen zufälligen Vektor generieren.

Nun können wir eine Einbettungsschicht mit vortrainierten Gewichten definieren:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Hinweis**: Beachten Sie, dass wir `trainable=False` setzen, wenn wir die `Embedding` erstellen. Das bedeutet, dass wir die Embedding-Schicht nicht neu trainieren. Dies kann dazu führen, dass die Genauigkeit etwas geringer ist, aber es beschleunigt das Training.

### Verwendung des Embedding-Vokabulars

Ein Problem bei dem vorherigen Ansatz ist, dass die in der TextVectorization und Embedding verwendeten Vokabulare unterschiedlich sind. Um dieses Problem zu lösen, können wir eine der folgenden Lösungen verwenden:
* Das Word2Vec-Modell mit unserem Vokabular neu trainieren.
* Unser Dataset mit dem Vokabular aus dem vortrainierten Word2Vec-Modell laden. Die Vokabulare, die zum Laden des Datasets verwendet werden, können während des Ladens angegeben werden.

Der zweite Ansatz scheint einfacher zu sein, also setzen wir ihn um. Zunächst erstellen wir eine `TextVectorization`-Schicht mit dem angegebenen Vokabular, das aus den Word2Vec-Embeddings stammt:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Die Gensim-Wort-Einbettungsbibliothek enthält eine praktische Funktion, `get_keras_embeddings`, die automatisch die entsprechende Keras-Einbettungsschicht für Sie erstellt.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Einer der Gründe, warum wir keine höhere Genauigkeit sehen, ist, dass einige Wörter aus unserem Datensatz im vortrainierten GloVe-Vokabular fehlen und daher im Wesentlichen ignoriert werden. Um dies zu überwinden, können wir unsere eigenen Einbettungen basierend auf unserem Datensatz trainieren.


## Kontextuelle Einbettungen

Eine zentrale Einschränkung traditioneller vortrainierter Einbettungsrepräsentationen wie Word2Vec ist die Tatsache, dass sie zwar eine gewisse Bedeutung eines Wortes erfassen können, aber nicht zwischen verschiedenen Bedeutungen unterscheiden können. Dies kann in nachgelagerten Modellen zu Problemen führen.

Zum Beispiel hat das Wort „play“ in diesen beiden Sätzen unterschiedliche Bedeutungen:
- Ich war in einem **Theaterstück** im Theater.
- John möchte mit seinen Freunden **spielen**.

Die vortrainierten Einbettungen, über die wir gesprochen haben, repräsentieren beide Bedeutungen des Wortes „play“ in derselben Einbettung. Um diese Einschränkung zu überwinden, müssen wir Einbettungen basierend auf dem **Sprachmodell** erstellen, das auf einem großen Textkorpus trainiert wurde und *versteht*, wie Wörter in unterschiedlichen Kontexten zusammengefügt werden können. Die Diskussion über kontextuelle Einbettungen liegt außerhalb des Umfangs dieses Tutorials, aber wir werden darauf zurückkommen, wenn wir im nächsten Abschnitt über Sprachmodelle sprechen.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
